In [1]:
from pathlib import Path
import re

import torch
import numpy as np
import joblib

In [2]:
path = Path("../data/cache/lm_douban/")
model = torch.load(path / "sentiment_model.pth")

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
model = model.cpu()
model.reset()
model = model.eval()

In [4]:
mapping = joblib.load("../data/mapping.pkl")

In [5]:
UNK = 2
def get_prediction(texts):
    input_tensor = torch.from_numpy(np.array([1] + [mapping.get(x, UNK-1) + 1 for x in texts])).long().unsqueeze(1)
    return model(input_tensor)[0].data.cpu().numpy()[0, 0]

In [6]:
get_prediction("看了快一半了才发现是mini的广告")

2.3815765

## ELI5 (LIME)

In [7]:
!pip install eli5 jieba

In [8]:
UNK = 2
def get_proba(rows):
    """eli5 only supports classifier, so we need to do some transformations."""
    probs = []
    for texts in rows:
        model.reset()
        texts = re.sub(r"\s+", "", texts)
        texts.replace("qqq", " ")
        input_tensor = torch.from_numpy(
            np.array([1] + [mapping.get(x, UNK-1) + 1 for x in texts])
        ).long().unsqueeze(1)
        pos = (np.clip(model(input_tensor)[0].data.cpu().numpy()[0, 0], 1, 5) - 1) / 4
        probs.append(np.array([1-pos, pos]))
    return np.stack(probs, axis=0)

In [9]:
import jieba
seg_list = jieba.cut("看了快一半了才发现是mini的广告", cut_all=False)
list(seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.430 seconds.
Prefix dict has been built succesfully.


['看', '了', '快', '一半', '了', '才', '发现', '是', 'mini', '的', '广告']

### Example 1

In [10]:
get_proba(["看 了 快 一半 了 才 发现 是 mini 的 广告"])

array([[0.65461, 0.34539]])

In [11]:
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut("看了快一半了才发现是mini的广告", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [12]:
te.metrics_

{'mean_KL_divergence': 0.006262968039355961, 'score': 0.8745423116722344}

In [13]:
te.samples_[:10]

['看 了  一半  才 发现 是 mini 的 ',
 ' 了 快     是   ',
 '   一半  才  是  的 ',
 '看 了  一半  才  是 mini 的 广告',
 '看   一半  才  是   广告',
 '看  快 一半 了   是 mini  广告',
 '          ',
 ' 了 快  了 才 发现  mini  ',
 '          ',
 '看 了   了  发现   的 广告']

#### Character-based Whitebox

In [14]:
te = TextExplainer(random_state=42, n_samples=5000, char_based=True)
te.fit("看了快一半了才发现是mini的广告", get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [15]:
te.metrics_

{'mean_KL_divergence': 0.006582223516253523, 'score': 0.8113948371442119}

In [16]:
te.samples_[:10]

['了一半发是ni广告',
 '看一半了才是min广',
 'm',
 '看了一半了现ii的广告',
 '了快是i的广',
 '看快半了才现是ini的告',
 '广',
 '看了快一半了才的广告',
 '了快一半了发现是mn的广',
 '看半了mn广']

### Example 2

In [17]:
te = TextExplainer(random_state=42, n_samples=2000)
te.fit(" ".join(jieba.cut("妈蛋，简直太好看了。最后的 DJqqqbattle 部分，兴奋的我，简直想从座位上站起来一起扭", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [18]:
te.samples_[:10]

['妈蛋 ， 简直 太 好看 了 。 最后 的      部分 ， 兴奋  我 ， 简直   座位  站 起来  ',
 '妈蛋 ， 简直 太 好看 了 。  的   DJqqqbattle   部分 ，  的 我 ， 简直  从 座位 上 站 起来 一起 扭',
 ' ， 简直    。 最后        ， 兴奋 的  ，    座位 上    扭',
 '妈蛋 ， 简直  好看  。  的   DJqqqbattle   部分 ， 兴奋 的  ， 简直    上  起来  ',
 ' ， 简直 太   。     DJqqqbattle    ，  的  ，    座位     ',
 '妈蛋 ，  太   。 最后        ， 兴奋   ，  想       ',
 '妈蛋 ， 简直   了 。 最后       部分 ， 兴奋 的 我 ， 简直 想    站   ',
 ' ， 简直  好看  。 最后 的       ，  的  ， 简直     站   扭',
 ' ， 简直 太 好看 了 。 最后 的       ， 兴奋 的 我 ，   从 座位 上 站 起来 一起 ',
 '妈蛋 ，   好看 了 。         ， 兴奋   ， 简直 想    站  一起 扭']

In [19]:
te.metrics_

{'mean_KL_divergence': 0.007374620790416627, 'score': 0.9685526136747798}

### Example 3

In [20]:
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut("十亿元很难花掉吗？投资一部《阿修罗》，瞬间就赔光了啊。这设定太没说服力了。", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [21]:
get_proba([" ".join(jieba.cut("十亿元很难花掉吗？投资一部《阿修罗》，瞬间就赔光了啊。这设定太没说服力了。", cut_all=False))])

array([[0.70298, 0.29702]])

In [22]:
te.metrics_

{'mean_KL_divergence': 0.0035649766958844153, 'score': 0.9601349095944697}

### Example 4

In [23]:
texts = (
    "爱上了让雷诺。爱上了这个意大利杀手的温情。爱上了他脸部轮廓，和刚毅的线条。"
    "“人生诸多辛苦，是不是只有童年如此。”玛蒂尔达问。里昂说，“一直如此。”这样的话，击中了内心深处。"
)
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [24]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.22752, 0.77248]])

In [25]:
te.metrics_

{'mean_KL_divergence': 0.007372524737682906, 'score': 0.9973906779376904}

### Example 5

In [26]:
texts = "看到泪飙，世上最温情的杀手。 幸运的姑娘在12岁就遇到了真正的男人。"
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [27]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.13664, 0.86336]])

In [28]:
te.metrics_

{'mean_KL_divergence': 0.007161345818982022, 'score': 0.9906848314283919}

### Example 6

In [29]:
texts = (
    "当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，"
    "这部电影在越来越多的人们心中的地位已超越了《阿甘》。每当现实令我疲惫得产生无力感，"
    "翻出这张碟，就重获力量。毫无疑问，本片位列男人必看的电影前三名！回顾那一段经典台词："
    "“有的人的羽翼是如此光辉，即使世界上最黑暗的牢狱，也无法长久地将他围困！”"
)
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [30]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.14933, 0.85067]])

In [31]:
te.metrics_

{'mean_KL_divergence': 0.009818931571126503, 'score': 0.9984571619034786}